In [35]:
#2
import scipy.stats as sps
import numpy as np

def t(s):
    n = len(s)
    return np.sqrt(n)*(s.mean() - 1)/s.std(ddof=1)

def mk(samples):
    p = []
    c_a = sps.norm.ppf(0.95)
    for sample in samples:
        n = len(sample)
        pr = t(sample) > c_a
        p.append(int(pr))
    p = np.array(p)
    
    return p.mean()

samples = sps.expon.rvs(size = (10000, 30), random_state = 0)
mk(samples)

0.0249

In [ ]:
#1
# код, читающий выборку, подаваемую на вход
import numpy as np
import scipy.stats as sps

sample = np.array(list(map(float, input().strip().split())))
def t(s):
    n = len(s)
    return np.sqrt(n) * s.mean() / s.std(ddof=1)

n = len(sample)
ans = 2 * sps.t(n - 1).sf(abs(t(sample)))
print(ans)


In [39]:
#3
import scipy.stats as sps
import numpy as np

def t(s):
    n = len(s)
    return np.sqrt(n)*(s.mean() - 1)/s.std(ddof=1)

def mk(samples):
    p = []
    c_a = sps.norm.ppf(0.95)
    for sample in samples:
        n = len(sample)
        pr = t(sample)
        p.append(pr)
    p = np.array(p)
    
    return np.quantile(p, 0.95)

samples = sps.expon.rvs(size = (100000, 30), random_state = 0)
mk(samples)

1.4133597507366644

In [52]:
#4

import scipy.stats as sps
import numpy as np
import scipy

c_a = 0.75672765 - 0.34059634

def make_samples():
    sample_sizes = sps.geom.rvs(0.55, size = 1000)
    samples = []
    for size in sample_sizes:
        samples.append(sps.bernoulli.rvs(0.5, size=size))
    return samples
    
def func_sample(sample):   
    wins, losses, go_w, go_l = 0, 0, 0, 0
    for game in sample:
        wins += sum(game)
        losses += len(game) - sum(game)
        if game[-1] == 0:
            go_l += 1
        else:
            go_w += 1

    def func(x):
        p_w, p_go_w, p_go_l = x
        res = (np.log(p_w) * wins 
           + np.log(1 - p_w) * losses 
           + np.log(p_go_l) * go_l
           + np.log(1 - p_go_l) * (losses - go_l)
           + np.log(p_go_w) * go_w
           + np.log(1 - p_go_w) * (wins - go_w))
        return -res
    return func

n = 300

s_samples = [make_samples() for i in range(n)]

bounds = [(1e-5, 1-1e-5), (1e-5, 1-1e-5), (1e-5, 1-1e-5)]

mk = []
for s in s_samples:
    res = scipy.optimize.minimize(func_sample(s), (0.5, 0.5, 0.5), bounds=bounds)
    mk.append(int(res.x[2] - res.x[1] > c_a))

np.array(mk).mean()

0.0

In [121]:
#5
import scipy.stats as sps

def t1(x, y):
    return np.corrcoef(np.array([x, y]))[0][1]

def t2(x, y):
    return t1(sps.rankdata(x), sps.rankdata(y))

def samples(rho, n):
    x, y = sps.multivariate_normal(cov=[[1, rho], [rho, 1]]).rvs(size=(n, 30), random_state=0).T
    return x, y
n = 10000

x_0, y_0 = samples(0, n)

c_a = np.quantile(np.array([t1(x, y) for x, y in zip(x_0.T, y_0.T)]), 0.95)

x_05, y_05 = samples(0.5, n)
prob1 = [int(t1(x, y) > c_a) for x, y in zip(x_05.T, y_05.T)]


c_a = np.quantile(np.array([t2(x, y) for x, y in zip(x_0.T, y_0.T)]), 0.95)
prob2 = [int(t2(x, y) > c_a) for x, y in zip(x_05.T, y_05.T)]

np.array(prob1).mean(), np.array(prob2).mean()

(0.8922, 0.8585)